In [2]:
import time
import mysql.connector
from neo4j import GraphDatabase

# ฟังก์ชันสำหรับเชื่อมต่อกับ MySQL
def connect_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Wealth3visual%",
        database="financials"
    )

# ฟังก์ชันสำหรับเชื่อมต่อกับ Neo4j
def connect_neo4j():
    uri = "bolt://localhost:7687"
    return GraphDatabase.driver(uri, auth=("neo4j", "Neo4jpassword"))

# ฟังก์ชันสำหรับการสืบค้นใน MySQL
def query_mysql(cursor, query):
    start_time = time.time()
    cursor.execute(query)
    results = cursor.fetchall()
    end_time = time.time()
    return results, end_time - start_time

# ฟังก์ชันสำหรับการสืบค้นใน Neo4j
def query_neo4j(driver, query):
    with driver.session() as session:
        start_time = time.time()
        results = session.run(query).data()
        end_time = time.time()
    return results, end_time - start_time

# ฟังก์ชันเปรียบเทียบเวลาการสืบค้นและความแม่นยำ
def compare_databases(neo4j_query_str,mysql_query_str):
    mysql_conn = connect_mysql()
    mysql_cursor = mysql_conn.cursor()

    neo4j_driver = connect_neo4j()

    # Query MySQL
    mysql_results, mysql_time = query_mysql(mysql_cursor, mysql_query_str)
    print(f"MySQL Results: {mysql_results}")
    print(f"MySQL Time: {mysql_time} seconds")

    # Query Neo4j
    neo4j_results, neo4j_time = query_neo4j(neo4j_driver, neo4j_query_str)
    print(f"Neo4j Results: {neo4j_results}")
    print(f"Neo4j Time: {neo4j_time} seconds")

    # เปรียบเทียบผลลัพธ์
    if mysql_results == neo4j_results:
        print("Results are identical")
    else:
        print("Results differ")

    # ปิดการเชื่อมต่อ
    mysql_conn.close()
    neo4j_driver.close()

    return mysql_time, neo4j_time

In [9]:
def main():
    # ตัวอย่างคำถาม: "อัตราส่วนการคืนทุน (ROE) ของบริษัท [ADVANC] ในไตรมาสที่ 1 ของปี 2019 คือเท่าไหร่?"
    mysql_query_str = """
    SELECT roe 
    FROM financial_statements
    WHERE symbol = 'ADVANC' AND year = 2019 AND quarter = 1;
    """
    
    neo4j_query_str = """
    MATCH (c:Company {symbol: 'ADVANC'})-[:HAS_FINANCIAL_STATEMENT]->(fs:FinancialStatement {year: '2019', quarter: '1'})
    MATCH (fs)-[:HAS_RATIO]->(r:FinancialRatio {type: 'ROE'})
    RETURN r.value AS ROE
    """

    neo4j_time,mysql_time  = compare_databases(neo4j_query_str,mysql_query_str)
    print(f"Time Comparison - MySQL: {mysql_time}s, Neo4j: {neo4j_time}s")

if __name__ == "__main__":
    main()


MySQL Results: [(Decimal('55.99'),)]
MySQL Time: 0.003021717071533203 seconds
Neo4j Results: [{'ROE': 55.99240704846936}]
Neo4j Time: 2.0770394802093506 seconds
Results differ
Time Comparison - MySQL: 2.0770394802093506s, Neo4j: 0.003021717071533203s


In [19]:
import time
from neo4j import GraphDatabase

# ฟังก์ชันสำหรับเชื่อมต่อกับ Neo4j
def connect_neo4j():
    uri = "bolt://localhost:7687"
    return GraphDatabase.driver(uri, auth=("neo4j", "Neo4jpassword"))

# ฟังก์ชันสำหรับการสืบค้นใน Neo4j
def query_neo4j(driver, query):
    with driver.session() as session:
        start_time = time.time()
        results = session.run(query).data()
        end_time = time.time()
    return results, end_time - start_time

# ฟังก์ชันเปรียบเทียบเวลาการสืบค้นและความแม่นยำ
def compare_databases(neo4j_query_str):

    neo4j_driver = connect_neo4j()

    # Query Neo4j
    neo4j_results, neo4j_time = query_neo4j(neo4j_driver, neo4j_query_str)
    print(f"Neo4j Results: {neo4j_results}")
    print(f"Neo4j Time: {neo4j_time} seconds")

    neo4j_driver.close()

    return  neo4j_time

In [22]:
def main():
    neo4j_query_str = """
    MATCH (c:Company {symbol: 'ADVANC'})-[:HAS_FINANCIAL_STATEMENT]->(fs2019:FinancialStatement {year: '2022', quarter: '1'})
    MATCH (fs2019)-[:HAS_RATIO]->(r2019:FinancialRatio {type: 'ROE'})
    MATCH (c)-[:HAS_FINANCIAL_STATEMENT]->(fs2023:FinancialStatement {year: '2023', quarter: '1'})
    MATCH (fs2023)-[:HAS_RATIO]->(r2023:FinancialRatio {type: 'ROE'})
    RETURN r2019.value AS ROE2019, r2023.value AS ROE2023
    """

    neo4j_time = compare_databases(neo4j_query_str)
    print(f"Neo4j: {neo4j_time}s")

if __name__ == "__main__":
    main()


Neo4j Results: [{'ROE2019': 36.140498267470385, 'ROE2023': 34.039870187412575}]
Neo4j Time: 2.0640342235565186 seconds
Neo4j: 2.0640342235565186s
